In [1]:
from kalah import *
# from pebblereaper import *
from extrakalah import *
depth = 3
weights = [0.51, 0.01, 0.34, 0.1]
h_list = [pit_total,side_total,chance_to_clear_right_most,congregate_seeds]
full_h_list = [pit_total,side_total,chance_to_clear_right_most,congregate_seeds,capture_defense]

class Jogador():
    def __init__(self, nome, fun):
        self.nome = nome
        self.fun = fun
    def display(self):
        print(self.nome+" ")
        
class JogadorAlfaBeta(Jogador):
    def __init__(self, nome, depth,fun_eval):
        self.nome = nome
        self.fun = lambda game, state: alphabeta_cutoff_search_new(state,game,depth,eval_fn=fun_eval)


##########  para ser independente dos jogos deveria devolver um método em string ou um atributo
def joga11(game, jog1, jog2,verbose=False):
    ### jog1 e jog2 são jogadores com funções que dado um estado do jogo devolvem a jogada que escolheram
    ### devolve o par de jogadores, a lista de jogadas e o resultado
    estado=game.initial
    proxjog = jog1
    lista_jogadas=[]
    lance = 0
    while not game.terminal_test(estado):
        if verbose:
            print('----------   LANCE:',lance)
            game.display(estado)
        jogada = proxjog.fun(game, estado) # state produced
        if verbose:
            print('JOGADA=',jogada)
            print("pit_total:",pit_total(estado,proxjog))
            print("side_total: ", side_total(estado,proxjog))
            print("chance_to_clear_right_most:",chance_to_clear_right_most(estado,proxjog))
            print("congregate_seeds: ", congregate_seeds(estado,proxjog))
            print("heuristic total:",linearPond(estado,proxjog,weights,h_list,100))
        estado=game.result(estado,jogada)
        lista_jogadas.append(jogada)
        proxjog = jog2 if proxjog == jog1 else jog1
        lance+=1
    #p jogou e ganhou
    util=game.utility(estado,0)
    if util == 1:
        resultado=jog1.nome
    elif util== -1:
        resultado = jog2.nome
    else:
        resultado='Empate'
    return ((jog1.nome,jog2.nome),lista_jogadas,resultado)


## Create Chapiteau

In [2]:
def toes(state,player):
    board = state.state
    if state.is_game_over():
        aux = state.result()
        return aux*100 if player == state.SOUTH else aux*-100 
    
    own_seeds = sum(board[:7]) if player == 0 else sum(board[7:14])
    opponent_seeds = sum(board[7:14]) if player == 0 else sum(board[:7])
    score = own_seeds - opponent_seeds
    return score

toe1 = JogadorAlfaBeta("chapiteau",depth,toes)
# toe2 = JogadorAlfaBeta("heehoo",6,toes)



## Create Random Player

In [3]:
def f_caos_intel(estado,jogador):
    """Quando é terminal: +100 para vitória, -100 para a derrota e 0 para o empate.
       Quando o tabuleiro é não terminal devolve 0, o que quer dizer que como o minimax baralha as acções, será random"""
    if estado.is_game_over():
        aux = estado.result()
        return aux*100 if jogador == estado.SOUTH else aux*-100
    return 0

el_caos_int6=JogadorAlfaBeta("El Caos Inteligente 6",depth,f_caos_intel)

# Pebbles!

In [13]:
import copy
# pebs = JogadorAlfaBeta('Pebbles',depth,Jogador_42)
# nome, depth,weights,features,win_value
h_list = [pit_total,side_total,chance_to_clear_right_most,congregate_seeds]
pebs =JogadorLinearPond('Pebbles',depth,weights,h_list,100)

# pebs2: more ags trained against chapiteau
# weights2 = copy.copy(weights)
# weights2.append(1)
weights2 = [0.45, 0.16, 0.05, 0.31, 0.24]
pebs2 = JogadorLinearPond('Pebbles2',depth,weights2,full_h_list,100)

# pebs3: more

# One game

In [5]:
mx = Kalah(0)
_,_,res=joga11(jogo,pebs2,pebs,True)
res

----------   LANCE: 0
----------------------
     _  _  _  _  _  _
     1  2  3  4  5  6
-------------------------
|  | 4| 4| 4| 3| 4| 4|  |
| 2|-----------------| 0|
|  | 3| 4| 4| 4| 4| 4|  | <--
-------------------------
     6  5  4  3  2  1

JOGADA= 2
pit_total: 2
side_total:  0
chance_to_clear_right_most: 0
congregate_seeds:  0.7999999999999998
heuristic total: 1.1
----------   LANCE: 1
----------------------
     _  _  _  _  _  _
     1  2  3  4  5  6
-------------------------
|  | 4| 4| 4| 3| 4| 4|  | <-- (Pass)
| 2|-----------------| 1|
|  | 3| 4| 0| 5| 5| 5|  |
-------------------------
     6  5  4  3  2  1

JOGADA= -1
pit_total: 1
side_total:  1
chance_to_clear_right_most: 0
congregate_seeds:  0.7999999999999998
heuristic total: 0.6
----------   LANCE: 2
----------------------
     _  _  _  _  _  _
     1  2  3  4  5  6
-------------------------
|  | 4| 4| 4| 3| 4| 4|  |
| 2|-----------------| 1|
|  | 3| 4| 0| 5| 5| 5|  | <--
-------------------------
     6  5  4  3  2  1



'Empate'

# N Match Pairs

In [14]:
scores={'Vitoria': 3, 'Empate': 1}

def traduzPontos(tabela):
    tabelaScore={}
    empates=tabela['Empate']
    for x in tabela:
        if x != 'Empate':
            tabelaScore[x]=scores['Vitoria']*tabela[x]+empates
    return tabelaScore

def getPoints(tabela, jogador):
    empates=tabela['Empate']
    total=0
    for x in tabela:
        total += scores['Vitoria']*tabela[x]+empates
    player_score = scores['Vitoria']*tabela[jogador]+empates
    return player_score/total

#ret_jog_index == 0 or 1 ONLY
def jogaNpares_GA(jogo,n,jog1,jog2,ret_jog_index):
    tabelaPrim={jog1.nome:0, jog2.nome:0, 'Empate':0}
    tabelaSeg={jog1.nome:0, jog2.nome:0, 'Empate':0}
    tabela={}
    for _ in range(n):
        _,_,vencedor=joga11(jogo,jog1,jog2)
        tabelaPrim[vencedor]+=1
        _,_,vencedor=joga11(jogo,jog2,jog1)
        tabelaSeg[vencedor]+=1
    for x in tabelaPrim:
        tabela[x]=tabelaPrim[x]+tabelaSeg[x]
    ret_jog = jog1.nome if ret_jog_index == 0 else jog2.nome
    return getPoints(tabela,ret_jog) # this is needed for GA

def jogaNpares(jogo,n,jog1,jog2,ret_jog_index):
    tabelaPrim={jog1.nome:0, jog2.nome:0, 'Empate':0}
    tabelaSeg={jog1.nome:0, jog2.nome:0, 'Empate':0}
    tabela={}
    for _ in range(n):
        _,_,vencedor=joga11(jogo,jog1,jog2)
        tabelaPrim[vencedor]+=1
        _,_,vencedor=joga11(jogo,jog2,jog1)
        tabelaSeg[vencedor]+=1
    for x in tabelaPrim:
        tabela[x]=tabelaPrim[x]+tabelaSeg[x]
    ret_jog = jog1.nome if ret_jog_index == 0 else jog2.nome
    # return getPoients(tabela,ret_jog) # this is needed for GA
    return tabelaPrim,tabelaSeg,tabela,traduzPontos(tabela) # this gives better data for individual npare runs

# SUPER IMPORTANT: if you want to see the full table using jogaNpares rather than the numerical win rate for GA, comment/uncomment the last two lines above

In [18]:
# jogaNpares()

jogo=Kalah(0)
#n jogaNpares(jogo,300,el_caos,el_caos_int6)
# jogo.result()

# _,_,res=joga11(jogo,pebs,pebs2,True)
jogaNpares(jogo,30,pebs,pebs2,1)
# print(res)


({'Pebbles': 27, 'Pebbles2': 3, 'Empate': 0},
 {'Pebbles': 27, 'Pebbles2': 2, 'Empate': 1},
 {'Pebbles': 54, 'Pebbles2': 5, 'Empate': 1},
 {'Pebbles': 163, 'Pebbles2': 16})

# SearchPlus Genetic Algorithm

In [12]:
from searchPlus import genetic_algorithm_custom
import random

# for steps in 0.01, use 2. 
DEPTH = depth 
PRECISION = 2
HEURISTICS = full_h_list
N_WEIGHTS = 5
TRAINING_ENEMY = toe1
N_GAMES = 30
N_GENERATION = 5
PMUT = 0.1
# generates n weights that adds up to 1, precision see above
# e.g. [0.7, 0.12, 0.12, 0.06]
def generate_population(n,precision):
    total = pow(10,precision)
    dividers = sorted(random.sample(range(1, total), n - 1))
    return [(a - b)/total for a, b in zip(dividers + [total], [0] + dividers)]

# generates a pool of potential changes from 0-1
# a list that goes [0.0, 0.01 ... 1] essentially. based on precision
def generate_pool(precision):
    return [round(x*pow(10,-precision),precision) for x in range(pow(10,precision))]
    
# generates 8 "gene" as an initial population
pop = [generate_population(N_WEIGHTS,PRECISION) for _ in range(8)]
pop_pool = generate_pool(PRECISION)
print(pop)
print(pop_pool)

# the fitness function evaluates how well a gene does. in this case we insert the weights
# into a player into npare games, which i've changed it to return the win percentage
pebs_generator = lambda name,w:JogadorLinearPond(name,DEPTH,w,HEURISTICS,100)
fitness= lambda name, w:jogaNpares_GA(jogo,N_GAMES,TRAINING_ENEMY,pebs_generator(name,w),1)

#call the genetic algorithm
best_fit_weights = genetic_algorithm_custom(pop,fitness,pop_pool,ngen=N_GENERATION,pmut=PMUT)

print(best_fit_weights)

[[0.08, 0.16, 0.26, 0.26, 0.24], [0.45, 0.23, 0.14, 0.05, 0.13], [0.31, 0.1, 0.24, 0.1, 0.25], [0.21, 0.16, 0.05, 0.31, 0.27], [0.1, 0.46, 0.09, 0.32, 0.03], [0.03, 0.06, 0.69, 0.2, 0.02], [0.05, 0.14, 0.41, 0.31, 0.09], [0.18, 0.28, 0.22, 0.14, 0.18]]
[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99]
([0.45, 0.16, 0.05, 0.31, 0.24], 0.7043010752688172)


# Tornament

In [ ]:
def incorpora(tabela,tx):
    for jog in tx:
        if jog not in tabela:
            tabela[jog]=tx[jog]
        else:
            tabela[jog]+=tx[jog]
    
def torneio(n,jogadores,jogo_id=0):
    jogo=Kalah(jogo_id)  # jogo gerado ao calha entre os "justos"
    tabela={}
    for i in range(len(jogadores)-1):
        jog1=jogadores[i]
        for j in range(i+1,len(jogadores)):
            jog2=jogadores[j]
            _,_,_,tabelaX = jogaNpares(jogo,n,jog1,jog2)
            incorpora(tabela,tabelaX)
    return tabela

In [ ]:
players = [pebs,el_caos_int6,toe1]